# Lab 9 - Structured Output - GPT-4o-mini

In [ ]:
from tqdm.notebook import tqdm
from datetime import datetime
from IPython.display import display, HTML
import numpy as np
import pickle
import openai
import json

#Don't include keys like this, use ENV vars!
with open('config.json') as fd:
    conf = json.loads(fd.read())
    gpt = openai.OpenAI(api_key=conf["openai_key"])

In [ ]:
def complete_function(prompt,function):
    resp = gpt.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant making decisions for routing tasks.",
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",
        temperature=0,
        tools=[
            {
                "type": "function",
                "function": function
            }
        ]
    )
    
    try:
        #Parse the JSON response
        content = resp.choices[0]
        tool_call = content.message.tool_calls[0]
        arguments_str = tool_call.function.arguments
        return json.loads(arguments_str)

    except (AttributeError, IndexError, TypeError, json.JSONDecodeError) as e:
        print(e)

    #Fallback to returning None if there was an error
    return None

## Yes or No (boolean true/false)

In [ ]:
def get_bool_prompt(query):
    return f"""# Instructions

Is the following question true or false?

## Question

{query}"""

In [ ]:
def get_bool_function():
    return {
        "name": "true_or_false",
        "strict": True,
        "parameters": {
          "type": "object",
          "properties": {
            "final_answer": {
              "type": "boolean"
            }
          },
          "required":["final_answer"],
          "additionalProperties":False
        }
    }

In [ ]:
def truefalse(question):
    prompt = get_bool_prompt(question)
    function = get_bool_function()
    response = complete_function(prompt,function)
    if response:
        print(f"""{response['final_answer']}\t<=\t{question}""")
    return response

In [ ]:
b = truefalse("Does 2+2 equal 4?")
b = truefalse("Does 2+2 equal 5?")
b = truefalse("Does the text 'Flower City AI conference at the Little Theatre in December' contain an event venue name?")
b = truefalse("Does the text 'Hello, Mr. Smith' list any attendees for an event?")
b = truefalse("Does the text 'Jane Doe is coming to Flower City AI' list any attendees for an event?")
b = truefalse("Does the text 'Jane Doe is coming to the Flower City AI conference' list any attendees for an event?")

## Pick from a list (enum)

In [ ]:
def get_classification_prompt(query):
    return f"""# Instructions

For an event management SaaS product, natural language queries need to be classified and routed to the appropriate branch. Classify the following user query:

## Query

{query}"""

In [ ]:
def get_classification_function():
    return {
        "name": "task_type",
        "description": "Classify the task based on the user query.",
        "strict": True,
        "parameters": {
          "type": "object",
          "properties": {
            "task_type": {
              "type": "string",
              "enum": ["create_event","list_events","create_attendee","list_attendees","other"]
            }
          },
          "required":["task_type"],
          "additionalProperties":False
        }
    }

In [ ]:
def classify(query):
    prompt = get_classification_prompt(query)
    function = get_classification_function()
    classification = complete_function(prompt,function)
    if classification:
        print(f"""{classification['task_type']}\t<=\t{query}""")
    return classification

In [ ]:
c = classify("What events are coming up?")
c = classify("Book the Ellison Lodge A for December 4th")
c = classify("Jane Doe is coming to the party on 12/4")
c = classify("Pencil in highland park gazebo for june 11th")
c = classify("Leah, Alice, and Fred are all coming on june 11th")
c = classify("Who is coming in June?")
c = classify("What spaces are available?")